In [1]:
from inner_iteration import inner_iteration
m = inner_iteration()

del D:/YURA/Aerokitties/OptFALT/polar.dat
D://Soft//XFOIL6.99//xfoil < D:/YURA/Aerokitties/OptFALT/commands.in
del D:/YURA/Aerokitties/OptFALT/polar.dat
D://Soft//XFOIL6.99//xfoil < D:/YURA/Aerokitties/OptFALT/commands.in
del D:/YURA/Aerokitties/OptFALT/polar.dat
D://Soft//XFOIL6.99//xfoil < D:/YURA/Aerokitties/OptFALT/commands.in
del D:/YURA/Aerokitties/OptFALT/polar.dat
D://Soft//XFOIL6.99//xfoil < D:/YURA/Aerokitties/OptFALT/commands.in
del D:/YURA/Aerokitties/OptFALT/polar.dat
D://Soft//XFOIL6.99//xfoil < D:/YURA/Aerokitties/OptFALT/commands.in
del D:/YURA/Aerokitties/OptFALT/polar.dat
D://Soft//XFOIL6.99//xfoil < D:/YURA/Aerokitties/OptFALT/commands.in
del D:/YURA/Aerokitties/OptFALT/polar.dat
D://Soft//XFOIL6.99//xfoil < D:/YURA/Aerokitties/OptFALT/commands.in
del D:/YURA/Aerokitties/OptFALT/polar.dat
D://Soft//XFOIL6.99//xfoil < D:/YURA/Aerokitties/OptFALT/commands.in
del D:/YURA/Aerokitties/OptFALT/polar.dat
D://Soft//XFOIL6.99//xfoil < D:/YURA/Aerokitties/OptFALT/commands.in
d